In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.transforms import Compose, Resize
import segmentation_models_pytorch as smp

from src.models.BaselineModel import BaselineModel
from src.evaluation.evaluate_result import evaluate_result
from src.datasets.DubaiSemanticSegmentationDataset import (
    DubaiSemanticSegmentationDataset,
)

from src.datasets.utils.ResizeToDivisibleBy32 import ResizeToDivisibleBy32

d:\__repos\aerial_segmentation\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Prepare environment

In [2]:
torch.cuda.is_available()

True

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
VAL_SIZE = 0.2
BATCH_SIZE = 1
SEED = 42
DUBAI_DATASET_PATH = "data/DubaiSemanticSegmentationDataset"
IMAGE_SIZE = 576

In [5]:
train_dataset = DubaiSemanticSegmentationDataset(
    DUBAI_DATASET_PATH,
    transforms=[Compose([Resize(IMAGE_SIZE), ResizeToDivisibleBy32()])],
)
print(len(train_dataset))

72


## Sanity check data

In [6]:
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

In [7]:
for images, masks in train_loader:
    print(images.shape)
    print(masks.shape)
    break

torch.Size([1, 3, 640, 576])
torch.Size([1, 1, 640, 576])


In [8]:
to_pil_transform = transforms.ToPILImage()

In [9]:
img = to_pil_transform(images.squeeze())

In [10]:
# img.show()

In [11]:
msk = to_pil_transform(masks.squeeze())

In [12]:
# msk.show()

# UNET

## Run example model on single image

In [13]:
model = smp.Unet(
    encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=6,  # model output channels (number of classes in your dataset)
    activation="softmax",
).to(device)

In [14]:
baseline_model = BaselineModel(classes=6).to(device)

In [15]:
torch.cuda.empty_cache()

In [16]:
model.eval()
with torch.no_grad():
    for images, masks in train_loader:
        print(images.shape)
        print(masks.shape)
        # (batch_size, channels, height, width)
        # (B, C, H, W)
        break
    output = model(images.to(device))
    output = torch.argmax(output, dim=1)

    output_baseline = baseline_model(images.to(device))
    output_baseline = torch.argmax(output_baseline, dim=1)

torch.Size([1, 3, 576, 640])
torch.Size([1, 1, 576, 640])


d:\__repos\aerial_segmentation\venv\lib\site-packages\torch\nn\modules\module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


In [17]:
print(masks.max())

tensor(5, dtype=torch.uint8)


In [18]:
print(masks.min())

tensor(0, dtype=torch.uint8)


In [19]:
print(output.shape)

torch.Size([1, 576, 640])


In [20]:
print(output_baseline.shape)

torch.Size([1, 576, 640])


In [21]:
unique, counts = np.unique(output.cpu(), return_counts=True)
print(dict(zip(unique, counts)))

{0: 32220, 1: 187, 2: 649, 3: 2808, 4: 5445, 5: 327331}


In [22]:
unique, counts = np.unique(masks.cpu(), return_counts=True)
print(dict(zip(unique, counts)))

{0: 127, 1: 114852, 2: 4966, 3: 591, 4: 247308, 5: 796}


In [23]:
# FIXME: wykonanie tego bez błędu == tensory mają takie same wymiary
evaluate_result(output, masks, mode="multiclass", num_classes=6)

{'iou': tensor(0.0056),
 'f1': tensor(0.0112),
 'accuracy': tensor(0.6704),
 'recall': tensor(0.0112)}

In [24]:
evaluate_result(output_baseline, masks, mode="multiclass", num_classes=6)

{'iou': tensor(0.0907),
 'f1': tensor(0.1663),
 'accuracy': tensor(0.7221),
 'recall': tensor(0.1663)}

## Run example model on dataset

In [25]:
model.eval()
metrics_dict = {
    "iou": [],
    "f1": [],
    "accuracy": [],
    "recall": [],
}

In [26]:
with torch.no_grad():
    for images, masks in train_loader:
        output = model(images.to(device))
        output = torch.argmax(output, dim=1)

        iter_metrics = evaluate_result(output, masks, mode="multiclass", num_classes=6)
        for key in metrics_dict.keys():
            metrics_dict[key].append(iter_metrics[key])

In [27]:
print(metrics_dict)

{'iou': [tensor(0.0153), tensor(0.0739), tensor(0.0127), tensor(0.0089), tensor(0.0278), tensor(0.0393), tensor(0.0026), tensor(0.0387), tensor(0.0409), tensor(0.0056), tensor(0.0561), tensor(0.0366), tensor(0.0317), tensor(0.0123), tensor(0.0083), tensor(0.0042), tensor(0.0191), tensor(0.0063), tensor(0.0550), tensor(0.0150), tensor(0.0024), tensor(0.1132), tensor(0.0035), tensor(0.0411), tensor(0.0291), tensor(0.0022), tensor(0.0036), tensor(0.0146), tensor(0.0112), tensor(0.0072), tensor(0.0172), tensor(0.0024), tensor(0.0030), tensor(0.0477), tensor(0.0046), tensor(0.0019), tensor(0.0098), tensor(0.0656), tensor(0.0063), tensor(0.0204), tensor(0.0150), tensor(0.0052), tensor(0.0212), tensor(0.0126), tensor(0.0089), tensor(0.0088), tensor(0.0094), tensor(0.0100), tensor(0.0069), tensor(0.0020), tensor(0.0045), tensor(0.0452), tensor(0.0056), tensor(0.0680), tensor(0.0005), tensor(0.0034), tensor(0.0026), tensor(0.0313), tensor(0.0157), tensor(0.0175), tensor(0.0058), tensor(0.0056),

# UNET++

## Run example model on single image

In [28]:
model = smp.UnetPlusPlus(
    encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=6,  # model output channels (number of classes in your dataset)
    activation="softmax",
).to(device)

In [29]:
baseline_model = BaselineModel()

In [30]:
torch.cuda.empty_cache()

In [31]:
model.eval()
with torch.no_grad():
    for images, masks in train_loader:
        print(images.shape)
        print(masks.shape)
        break
    output = model(images.to(device))
    output = torch.argmax(output, dim=1)

    output_baseline = baseline_model(images.to(device))
    output_baseline = torch.argmax(output_baseline, dim=1)

torch.Size([1, 3, 576, 640])
torch.Size([1, 1, 576, 640])


In [32]:
print(output.shape)

torch.Size([1, 576, 640])


In [33]:
print(output_baseline.shape)

torch.Size([1, 576, 640])


In [34]:
to_pil_transform = transforms.ToPILImage()

In [35]:
img = to_pil_transform(images.squeeze())

In [36]:
# img.show()

In [37]:
msk = to_pil_transform(masks.squeeze())

In [38]:
# msk.show()

In [39]:
unique, counts = np.unique(output.to('cpu'), return_counts=True)
print(dict(zip(unique, counts)))

{1: 1, 2: 4, 3: 367997, 4: 307, 5: 331}


In [40]:
outp = to_pil_transform(output.int().squeeze().cpu())

In [41]:
# plt.imsave("assets/aerial-drone-example-unet-plus-plus-output.jpeg", np.array(outp))

In [42]:
# outp.show()

In [43]:
evaluate_result(output, masks, mode="multiclass", num_classes=6)

{'iou': tensor(0.0897),
 'f1': tensor(0.1647),
 'accuracy': tensor(0.7216),
 'recall': tensor(0.1647)}

## DeepLabV3

## Run example model on single image

In [44]:
model = smp.DeepLabV3(
    encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=6,
    activation="softmax"
).to(device)

In [45]:
baseline_model = BaselineModel()

In [46]:
torch.cuda.empty_cache()

In [47]:
model.eval()
with torch.no_grad():
    for images, masks in train_loader:
        print(images.shape)
        print(masks.shape)
        break
    output = model(images.to(device))
    output = torch.argmax(output, dim=1)

    output_baseline = baseline_model(images.to(device))
    output_baseline = torch.argmax(output_baseline, dim=1)

torch.Size([1, 3, 576, 608])
torch.Size([1, 1, 576, 608])


In [48]:
print(output.shape)

torch.Size([1, 576, 608])


In [49]:
print(output_baseline.shape)

torch.Size([1, 576, 608])


In [50]:
to_pil_transform = transforms.ToPILImage()

In [51]:
img = to_pil_transform(images.squeeze())

In [52]:
# img.show()

In [53]:
msk = to_pil_transform(masks.squeeze())

In [54]:
# msk.show()

In [55]:
outp = to_pil_transform(output.int().squeeze().cpu())

In [56]:
plt.imsave("assets/aerial-drone-example-deep-lab-v3-output.jpeg", np.array(outp))

In [57]:
# outp.show()

In [58]:
evaluate_result(output, masks, mode="multiclass", num_classes=6)

{'iou': tensor(0.0883),
 'f1': tensor(0.1623),
 'accuracy': tensor(0.7208),
 'recall': tensor(0.1623)}

## DeepLabV3+

## Run example model on single image

In [59]:
model = smp.DeepLabV3Plus(
    encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=6,
    activation="softmax"
).to(device)

In [60]:
baseline_model = BaselineModel()

In [61]:
torch.cuda.empty_cache()

In [62]:
model.eval()
with torch.no_grad():
    for images, masks in train_loader:
        print(images.shape)
        print(masks.shape)
        break
    output = model(images.to(device))
    output = torch.argmax(output, dim=1)

    output_baseline = baseline_model(images.to(device))
    output_baseline = torch.argmax(output_baseline, dim=1)

torch.Size([1, 3, 576, 736])
torch.Size([1, 1, 576, 736])


In [63]:
print(output.shape)

torch.Size([1, 576, 736])


In [64]:
print(output_baseline.shape)

torch.Size([1, 576, 736])


In [65]:
to_pil_transform = transforms.ToPILImage()

In [66]:
img = to_pil_transform(images.squeeze())

In [67]:
# img.show()

In [68]:
msk = to_pil_transform(masks.squeeze())

In [69]:
# msk.show()

In [70]:
outp = to_pil_transform(output.int().squeeze().cpu())

In [71]:
plt.imsave("assets/aerial-drone-example-deep-lav-v3-plus-output.jpeg", np.array(outp))

In [72]:
# outp.show()

In [73]:
evaluate_result(output, masks, mode="multiclass", num_classes=6)

{'iou': tensor(0.0066),
 'f1': tensor(0.0131),
 'accuracy': tensor(0.6710),
 'recall': tensor(0.0131)}